In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load data from CSV
data = pd.read_csv("var_cars.csv")
print(data)

     mpg  cyl   disp   hp  drat     wt   qsec     vs     am   gear  carb  \
0   21.0    6  160.0  110  3.90  2.620  16.46  False   True   four     4   
1   21.0    6  160.0  110  3.90  2.875  17.02  False   True   four     4   
2   22.8    4  108.0   93  3.85  2.320  18.61   True   True   four     1   
3   21.4    6  258.0  110  3.08  3.215  19.44   True  False  three     1   
4   18.7    8  360.0  175  3.15  3.440  17.02  False  False  three     2   
5   18.1    6  225.0  105  2.76  3.460  20.22   True  False  three     1   
6   14.3    8  360.0  245  3.21  3.570  15.84  False  False  three     4   
7   24.4    4  146.7   62  3.69  3.190  20.00   True  False   four     2   
8   22.8    4  140.8   95  3.92  3.150  22.90   True  False   four     2   
9   19.2    6  167.6  123  3.92  3.440  18.30   True  False   four     4   
10  17.8    6  167.6  123  3.92  3.440  18.90   True  False   four     4   
11  16.4    8  275.8  180  3.07  4.070  17.40  False  False  three     3   
12  17.3    

In [2]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# List of categorical columns
categorical_cols = ["vs", "am", "gear", "country"]

# Apply Label Encoding
label_encoders = {}
for col in categorical_cols:
    label_encoders[col] = LabelEncoder()
    data[col] = label_encoders[col].fit_transform(data[col])

In [3]:
data = data.astype({'vs': 'category', 'am': 'category','gear':'category','country': 'category'})

In [4]:
print(data.dtypes)

mpg         float64
cyl           int64
disp        float64
hp            int64
drat        float64
wt          float64
qsec        float64
vs         category
am         category
gear       category
carb          int64
country    category
dtype: object


In [9]:
!pip install pygam

In [10]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neural_network import MLPRegressor as mlp
from sklearn.linear_model import GammaRegressor as gam
from pygam import LinearGAM, s, f

import pandas as pd
from sklearn.model_selection import train_test_split

# I have loaded your dataset into a DataFrame 'data'
X = data.drop(columns=['mpg'])  # Features
y = data['mpg']  # Target variable

# Optionally, split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_1 = LinearRegression()
# Train the model
model_1.fit(X_train, y_train)

model_2 = mlp(hidden_layer_sizes=5, activation="relu")
model_2.fit(X_train, y_train)

model_3 = gam()
model_3.fit(X_train, y_train)

model_4 = LinearGAM(s(0, n_splines=30)).fit(X, y)

# Example evaluation
train_score = model_4.score(X_train, y_train)
test_score = model_4.score(X_test, y_test)

print(f"Training R^2 score: {train_score:.4f}")
print(f"Testing R^2 score: {test_score:.4f}")

Training R^2 score: 0.7466
Testing R^2 score: 0.6890


/Users/taoyanling/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/taoyanling/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_glm/glm.py:282: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res)


In [13]:
!pip install scikit-explain

In [15]:
import skexplain
explainer = skexplain.ExplainToolkit(('LinearRegression', model_1), X=X, y=y,)

In [16]:
ale_1d_ds = explainer.ale(features='all', n_bootstrap=2, subsample=10000, n_jobs=1, n_bins=20)

ALE Categorical Features: 100%|███████████████████| 4/4 [00:01<00:00,  3.90it/s]


In [17]:
# Ensure the columns are properly categorized
data = data.astype({
    'vs': pd.CategoricalDtype(categories=['FALSE', 'TRUE']),
    'am': pd.CategoricalDtype(categories=['TRUE', 'FALSE']),
    'gear': pd.CategoricalDtype(categories=['three', 'four', 'five'], ordered=True),
    'country': pd.CategoricalDtype(categories=['Sweden', 'UK', 'Japan', 'Italy', 'Germany', 'USA'])
})

In [20]:
from pyale import pyale
ale_plot(data, target_col='mpg', ale_ds=ale_1d_ds, categorical_cols=['vs', 'am', 'gear', 'country'], centralization='median')

ModuleNotFoundError: No module named 'pyale'